# Set folder and seed

In [1]:
import os
os.chdir("..")  # Move one level up  
print(os.getcwd())

/Users/mapa/Documents/GitHub/Thesis_Neurodiversity


In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.options.mode.chained_assignment = None
meta_data = pd.read_csv(f'data.nosync/phenotypic/subjects_with_meta_{7}.csv',
                         index_col= 'Unnamed: 0')
meta_data['Sub ID'] = meta_data['Sub ID'].apply(lambda x: str(x).zfill(7))

#Get participants and their meta data
participants = meta_data.drop_duplicates(subset=['Sub ID', 'Dataset'])
participants['Co-Diagnosis'] = participants['Co-Diagnosis'].replace({np.nan: ''})
participants['Co-Diagnosis'] = participants['Co-Diagnosis'].apply(lambda x: '-'+x if x != '' else x)
participants['Diagnosis'] = participants['Diagnosis'] + participants['Co-Diagnosis']
participants['Diagnosis'] = participants['Diagnosis'].replace({'ASD-Other': 'ASD', 
                                                               'ADHD-Other': 'ADHD',
                                                               'TD-Other':'TD'})

to_remove = pd.read_csv('data.nosync/phenotypic/missing_metadata.csv')
to_remove = pd.concat([to_remove, pd.read_csv('data.nosync/stats/head_movement/motion_summary_all_subjects.csv')])
to_remove = to_remove[['Sub ID', 'Dataset', 'Exclude']]
to_remove = to_remove[to_remove['Exclude'] == True]
to_remove = to_remove.drop_duplicates(['Sub ID', 'Dataset'])
to_remove['Sub ID'] = to_remove['Sub ID'].apply(lambda x: str(x).zfill(7))

#Remove and select columns
participants['temp'] = list(zip(participants['Sub ID'],participants['Dataset']))
participants = participants[~participants['temp'].isin(list(zip(to_remove['Sub ID'],to_remove['Dataset'])))]
participants = participants[['Sub ID', 'Dataset', 'Age', 'Sex', 'IQ', 'Diagnosis']]


In [4]:
#Create bins
age_median = np.quantile(participants['Age'], 0.50)
iq_mean = 100

stratify_base = participants[['Age', 'Sex', 'IQ', 'Diagnosis', 'Sub ID', 'Dataset']].copy()
stratify_base['Age'] = stratify_base['Age'].apply(lambda x: 'q2' if x >= age_median
                                                            else 'q1') 
stratify_base['IQ'] = stratify_base['IQ'].apply(lambda x: 'q2' if x >= iq_mean
                                                            else 'q1')


train = []
test = []
val = []

priority = ['Diagnosis', 'Sex', 'Age', 'IQ']

for i in range(0,4):
    if i == 0:
        temp_prio = ['Diagnosis', 'Sex', 'Age', 'IQ']
    else:
        temp_prio = priority[:-i]
    
    #Get the groups and their count, and validation set size
    stratify_groups = stratify_base[temp_prio].groupby(temp_prio).value_counts().reset_index()
    stratify_groups['group'] = [i for i in range(len(stratify_groups))]
    stratify_groups['val_size'] = round(stratify_groups['count']*0.15,0)

    #Get those that can be stratified
    ready =  stratify_base.merge(stratify_groups[stratify_groups['count'] >= 3], 
                                 on=temp_prio, 
                                 how='inner')
    
    #Remove those that can be stratified
    stratify_base = stratify_base.merge(stratify_groups[stratify_groups['count'] < 3], 
                                    on=temp_prio, 
                                    how='inner')

    stratify_base = stratify_base[priority + ['Sub ID', 'Dataset']]
    
    #If less than three participants, stop and add to train
    if len(stratify_base) < 3:
        train.append(stratify_base)
        break
    
    #Make the split for the ready groups
    for idx, row in stratify_groups[stratify_groups['count'] >= 3].iterrows():
        ready_group = ready[ready['group'] == row['group']].reset_index()
        #Make index lists
        train_idx, test_idx, val_idx = [], [], []
        train_count, test_count, val_count = 0, 0, 0

        #Calculate test/val size
        test_size, val_size = round(len(ready_group)*0.15), round(len(ready_group)*0.15)
        
        for i in ready_group.index.to_list():
            if test_count < test_size:
                test_idx.append(i)
                test_count += 1
            elif val_count < val_size:
                val_idx.append(i)
                val_count += 1
            else:
                train_idx.append(i)
                train_count += 1
        train.append(ready_group[ready_group.index.isin(train_idx)])
        test.append(ready_group[ready_group.index.isin(test_idx)])
        val.append(ready_group[ready_group.index.isin(val_idx)])


In [5]:
train = pd.concat(train)[['Diagnosis', 'Sex', 'Age', 'IQ','Sub ID', 'Dataset']]
train_id = list(zip(train['Sub ID'],train['Dataset']))

val = pd.concat(val)[['Diagnosis', 'Sex', 'Age', 'IQ','Sub ID', 'Dataset']]
val_id = list(zip(val['Sub ID'],val['Dataset']))

test = pd.concat(test)[['Diagnosis', 'Sex', 'Age', 'IQ','Sub ID', 'Dataset']]
test_id = list(zip(test['Sub ID'],test['Dataset']))

In [6]:
nx_files = os.listdir('data.nosync/networks_multi')
if ".DS_Store" in nx_files:
    nx_files.remove(".DS_Store")
nx_files = pd.DataFrame(nx_files, columns=['file'])
nx_files['Sub ID'] = nx_files['file'].apply(lambda x: x.split('_')[0].zfill(7))
nx_files['Dataset'] = nx_files['file'].apply(lambda x: x.split('_')[2])
nx_files['temp'] = list(zip(nx_files['Sub ID'],nx_files['Dataset']))
nx_files['file'] = 'data.nosync/networks_multi/' + nx_files['file']

In [7]:
#save train
train = nx_files[nx_files['temp'].isin(train_id)]
train['file'].to_csv(f'data.nosync/networks_multi/train_set_files.csv', index= False)
train

,file,Sub ID,Dataset,temp
0,data.nosync/networks_multi/1320247_run-1_ADHD2...,1320247,ADHD200,"(1320247, ADHD200)"
1,data.nosync/networks_multi/8415034_run-2_ADHD2...,8415034,ADHD200,"(8415034, ADHD200)"
4,data.nosync/networks_multi/3011311_run-2_ADHD2...,3011311,ADHD200,"(3011311, ADHD200)"
5,data.nosync/networks_multi/0010087_run-2_ADHD2...,0010087,ADHD200,"(0010087, ADHD200)"
6,data.nosync/networks_multi/0010030_run-2_ADHD2...,0010030,ADHD200,"(0010030, ADHD200)"
...,...,...,...,...
651,data.nosync/networks_multi/0010115_run-1_ADHD2...,0010115,ADHD200,"(0010115, ADHD200)"
652,data.nosync/networks_multi/0010086_run-2_ADHD2...,0010086,ADHD200,"(0010086, ADHD200)"
654,data.nosync/networks_multi/1127915_run-1_ADHD2...,1127915,ADHD200,"(1127915, ADHD200)"
655,data.nosync/networks_multi/2136051_run-1_ADHD2...,2136051,ADHD200,"(2136051, ADHD200)"


In [8]:
val = nx_files[nx_files['temp'].isin(val_id)]
val['file'].to_csv(f'data.nosync/networks_multi/val_set_files.csv', index= False)
val

,file,Sub ID,Dataset,temp
3,data.nosync/networks_multi/0051038_run-1_ABIDE...,0051038,ABIDEI,"(0051038, ABIDEI)"
8,data.nosync/networks_multi/0051047_run-1_ABIDE...,0051047,ABIDEI,"(0051047, ABIDEI)"
23,data.nosync/networks_multi/0050991_run-1_ABIDE...,0050991,ABIDEI,"(0050991, ABIDEI)"
34,data.nosync/networks_multi/0021025_run-1_ADHD2...,0021025,ADHD200,"(0021025, ADHD200)"
40,data.nosync/networks_multi/0050968_run-1_ABIDE...,0050968,ABIDEI,"(0050968, ABIDEI)"
...,...,...,...,...
632,data.nosync/networks_multi/3441455_run-1_ADHD2...,3441455,ADHD200,"(3441455, ADHD200)"
635,data.nosync/networks_multi/1992284_run-1_ADHD2...,1992284,ADHD200,"(1992284, ADHD200)"
650,data.nosync/networks_multi/0050966_run-1_ABIDE...,0050966,ABIDEI,"(0050966, ABIDEI)"
659,data.nosync/networks_multi/0051012_run-1_ABIDE...,0051012,ABIDEI,"(0051012, ABIDEI)"


In [9]:
test = nx_files[nx_files['temp'].isin(test_id)]
test['file'].to_csv(f'data.nosync/networks_multi/test_set_files.csv', index= False)
test

,file,Sub ID,Dataset,temp
2,data.nosync/networks_multi/0021003_run-1_ADHD2...,0021003,ADHD200,"(0021003, ADHD200)"
10,data.nosync/networks_multi/0051086_run-1_ABIDE...,0051086,ABIDEI,"(0051086, ABIDEI)"
15,data.nosync/networks_multi/3433846_run-2_ADHD2...,3433846,ADHD200,"(3433846, ADHD200)"
17,data.nosync/networks_multi/2260910_run-1_ADHD2...,2260910,ADHD200,"(2260910, ADHD200)"
19,data.nosync/networks_multi/5971050_run-1_ADHD2...,5971050,ADHD200,"(5971050, ADHD200)"
...,...,...,...,...
611,data.nosync/networks_multi/0021046_run-1_ADHD2...,0021046,ADHD200,"(0021046, ADHD200)"
646,data.nosync/networks_multi/0021002_run-1_ADHD2...,0021002,ADHD200,"(0021002, ADHD200)"
656,data.nosync/networks_multi/0051030_run-1_ABIDE...,0051030,ABIDEI,"(0051030, ABIDEI)"
657,data.nosync/networks_multi/0029229_run-1_ABIDE...,0029229,ABIDEII,"(0029229, ABIDEII)"


# Get data set stats

### Train

In [10]:
train_list = pd.read_csv(f'data.nosync/networks_multi/train_set_files.csv')
train_list['participant'] = train_list['file'].apply(lambda x: x.split('/')[2])
train_list['participant'] = train_list['participant'].apply(lambda x: x.split('_')[0] + '-' + x.split('_')[2])
train_list['diagnosis'] = train_list['file'].apply(lambda x: x.split('_')[4])
train_list_scans = train_list[['participant', 'diagnosis']].groupby('diagnosis').count().reset_index()
train_list_scans['percent (scan)'] = train_list_scans['participant']/train_list_scans['participant'].sum()*100

train_list_participants = train_list.drop_duplicates('participant')[['participant', 'diagnosis']].groupby('diagnosis').count().reset_index()
train_list_participants['percent (perticipant)'] = train_list_participants['participant']/train_list_participants['participant'].sum()*100
train_list_participants['percent (scan)'] = train_list_scans['percent (scan)']
train_list_participants['scans'] = train_list_scans['participant']
train_list_participants[['diagnosis', 'participant', 'scans', 'percent (perticipant)', 'percent (scan)']]

,diagnosis,participant,scans,percent (perticipant),percent (scan)
0,ADHD,87,143,27.358491,32.426304
1,ASD,58,58,18.238994,13.151927
2,ASD-ADHD,23,23,7.232704,5.215420
3,TD,150,217,47.169811,49.206349


### Validation

In [11]:
val_list = pd.read_csv(f'data.nosync/networks_multi/val_set_files.csv')
val_list['participant'] = val_list['file'].apply(lambda x: x.split('/')[2])
val_list['participant'] = val_list['participant'].apply(lambda x: x.split('_')[0] + '-' + x.split('_')[2])
val_list['diagnosis'] = val_list['file'].apply(lambda x: x.split('_')[4])
val_list_scans = val_list[['participant', 'diagnosis']].groupby('diagnosis').count().reset_index()
val_list_scans['percent (scan)'] = val_list_scans['participant']/val_list_scans['participant'].sum()*100

val_list_participants = val_list.drop_duplicates('participant')[['participant', 'diagnosis']].groupby('diagnosis').count().reset_index()
val_list_participants['percent (perticipant)'] = val_list_participants['participant']/val_list_participants['participant'].sum()*100
val_list_participants['percent (scan)'] = val_list_scans['percent (scan)']
val_list_participants['scans'] = val_list_scans['participant']
val_list_participants[['diagnosis', 'participant', 'scans', 'percent (perticipant)', 'percent (scan)']]

,diagnosis,participant,scans,percent (perticipant),percent (scan)
0,ADHD,20,33,29.411765,40.740741
1,ASD,11,11,16.176471,13.580247
2,ASD-ADHD,5,5,7.352941,6.172840
3,TD,32,32,47.058824,39.506173


### Test

In [12]:
test_list = pd.read_csv(f'data.nosync/networks_multi/test_set_files.csv')
test_list['participant'] = test_list['file'].apply(lambda x: x.split('/')[2])
test_list['participant'] = test_list['participant'].apply(lambda x: x.split('_')[0] + '-' + x.split('_')[2])
test_list['diagnosis'] = test_list['file'].apply(lambda x: x.split('_')[4])
test_list_scans = test_list[['participant', 'diagnosis']].groupby('diagnosis').count().reset_index()
test_list_scans['percent (scan)'] = test_list_scans['participant']/test_list_scans['participant'].sum()*100

test_list_participants = test_list.drop_duplicates('participant')[['participant', 'diagnosis']].groupby('diagnosis').count().reset_index()
test_list_participants['percent (perticipant)'] = test_list_participants['participant']/test_list_participants['participant'].sum()*100
test_list_participants['percent (scan)'] = test_list_scans['percent (scan)']
test_list_participants['scans'] = test_list_scans['participant']
test_list_participants[['diagnosis', 'participant', 'scans', 'percent (perticipant)', 'percent (scan)']]

,diagnosis,participant,scans,percent (perticipant),percent (scan)
0,ADHD,20,29,29.411765,37.662338
1,ASD,11,11,16.176471,14.285714
2,ASD-ADHD,5,5,7.352941,6.493506
3,TD,32,32,47.058824,41.558442
